In [ ]:
### CRISPRDetect ###
#import CRISPRDetect output gff
detect_db=pd.read_csv(str(snakemake.input.crisprdetect), 
sep="\t",
names=["contig_id","tool","type","start","stop","length","strand","blank","add_info"])

#select columns
detect_db=detect_db[["contig_id","tool","type","start","stop","length","strand","add_info"]]

# split arrays and spacers
spacers = detect_db[detect_db["type"] == "binding_site"]
arrays = detect_db[detect_db["type"] == "repeat_region"]

# define array information
arrays[['id', 'seq', 'dbxref', 'ontology', 'score']] = arrays['add_info'].str.split(';', expand=True)
arrays['array_id'] = arrays['id'].str.split('=', expand=True)[1]
arrays['array_num'] = arrays['array_id'].str.split('_', expand=True)[0]
arrays["num"] = arrays['array_num'].str.split('CRISPR', expand=True)[1]
arrays['array_info'] = arrays['tool'] + '_' + arrays['contig_id'] + '_CRISPR_' + arrays['num'] +'_' + arrays["start"] + '_' + arrays["stop"]

# define spacer_information
spacers[['id', 'name', 'parent', 'seq', 'dbxref', 'ontology', 'score']] = spacers['add_info'].str.split(';', expand=True)
spacers['parent_split'] = spacers['parent'].str.split('=', expand=True)[1]
spacers['spacer_id'] = spacers['id'].str.split('=', expand=True)[1]
spacers['spacer_num'] = spacers['spacer_id'].str.split('_', expand=True)[1]
spacers["num"] = spacers['spacer_num'].str.split('SPACER', expand=True)[1]
spacers['spacer_info'] = 'SPACER' + '_' + spacers['num'] + '_' + spacers["start"] + '_' + spacers["stop"]
spacers['spacer_seq'] = spacers['seq'].str.split('=', expand=True)[1]
spacers.rename(columns={'parent_split':'array_id'}, inplace=True)

# combine dataframes
db_merged = spacers.merge(arrays, on='array_id')
db_merged['output_name'] = db_merged['array_info'] + '_' + db_merged['spacer_info']

#generate fasta
outfile=open(str(snakemake.output), 'w')

for index,row in db_merged.iterrows():
	outfile.write('>'+row['output_name']+'\n')
	outfile.write(row['spacer_seq']+'\n')

outfile.close()


### CRISPRidentify ###
updated = []

# read in the fasta
for record in SeqIO.parse(str(snakemake.input.crispridentify), "fasta"):
	record.id = 'CRISPRidentify_' + record.id
	updated.append(record)

SeqIO.write(updated, str(snakemake.output), "fasta")